In [2]:
import os
import sys
import cv2
import matplotlib.pyplot as plt
import numpy as np

face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

def detect_one_face(im):
    gray=cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        return (0,0,0,0)
    return faces[0]

#kalman filter
def kalman_tracker(v):
    # Open output file
    #output_name = sys.argv[3] + file_name
    #output = open(output_name,"w")

    frameCounter = 0
    # read first frame
    ret ,frame = v.read()
    if ret == False:
        return


    # detect face in first frame
    c,r,w,h = detect_one_face(frame)
    frameCounter = frameCounter + 1

    # set the initial tracking window
    track_window = (c,r,w,h)
    state = np.array([c+w/2,r+h/2,0,0], dtype='float64') # initial position
    kalman = cv2.KalmanFilter(4,2,0)	
    kalman.transitionMatrix = np.array([
                                    [1., 0., 0.1, 0.],
                                    [0., 1., 0., 0.1],
                                    [0., 0., 1., 0.],
                                    [0., 0., 0., 1.]])
    kalman.measurementMatrix = 1. * np.eye(2, 4)
    kalman.processNoiseCov = 1e-5 * np.eye(4, 4)
    kalman.measurementNoiseCov = 1e-3 * np.eye(2, 2)
    kalman.errorCovPost = 1e-1 * np.eye(4, 4)
    kalman.statePost = state
    measurement = np.array([c+w/2, r+h/2], dtype='float64')
   
    
    while(1):
        ret ,frame = v.read() # read another frame
        if ret == False:
            break
        
        prediction = kalman.predict() #prediction
        x,y,w,h = detect_one_face(frame) #checking measurement
        
        ww = w
        hh = h
        
        img_measurement = cv2.rectangle(frame,(x,y),(x+w ,y+h),(255,0,0),2)
        cv2.putText(img_measurement,'1_mea',(int(x),int(y+h)), cv2.FONT_HERSHEY_SIMPLEX, 2, (30, 90, 2),cv2.LINE_AA)
        
        measurement = np.array([x+w/2, y+h/2], dtype='float64')
            
        if not (x ==0 and y==0 and w==0 and h ==0):
            posterior = kalman.correct(measurement)
        if x ==0 and y==0 and w==0 and h ==0:
            x,y,w,h = prediction
        else:
            x,y,w,h = posterior	

        img2 = cv2.rectangle(frame, (int(x - ww/2),int(y - hh/2)), (int(x+ww/2),int(y+hh/2)), (0,0,255),1)
        cv2.putText(img2,'1',(int(x),int(y+h)), cv2.FONT_HERSHEY_SIMPLEX, 2, (30, 90, 2),cv2.LINE_AA)
        cv2.imshow('img2',img2)
        k = cv2.waitKey(1) & 0xff
        if k == 27: # nhấn nút Esc để thoát
            break
        frameCounter = frameCounter + 1
        #print("frame: ", frameCounter)
    v.release()
    cv2.destroyAllWindows()

    #output.close()
video = cv2.VideoCapture("test3.mp4");
kalman_tracker(video)